# Workshop #2. Expression quantification

Firstly, let's create set of transcripts with different elements that would generate reads with length equal to one letter. Also let's create some random number of each transcript's molecules.

In [1]:
import random
import numpy as np

seed = 10

transcripts = [
    "ABCDEZQ",
    "DEFHI",
    "GHIL",
    "KLONPQST",
    "PQRSTUW",
]

random.seed(seed)
n_molecules = [random.randint(30, 1000) for i in transcripts]
abundance_ground_truth = np.array(
    [n_molecules[i] / sum(n_molecules) for i in range(len(n_molecules))]
)

print("\n".join([
    f"{transcripts[i]}: {n_molecules[i]} ({(abundance_ground_truth[i] * 100):.2f}%)"
    for i in range(len(n_molecules))
]))

ABCDEZQ: 615 (26.83%)
DEFHI: 63 (2.75%)
GHIL: 469 (20.46%)
KLONPQST: 524 (22.86%)
PQRSTUW: 621 (27.09%)


Next, let's generate random reads from these transcripts.

In [2]:
frac_reads = 0.1

random.seed(seed)
letters_pool = "".join([
    transcripts[i] * n_molecules[i]
    for i in range(len(transcripts))
])
reads = random.sample(letters_pool, int(frac_reads * len(letters_pool)))
print(f"We have {frac_reads * 100}% of transcripts in our pool ({len(reads)})")

We have 10.0% of transcripts in our pool (1503)


So, let's continue with the EM algorithm to estimate original abundance of different transcripts.

In [3]:
def EM(transcripts, reads):
  # Works for an example with k = 1 and read_length = 1 — the simplest case

  length = np.array([len(i) for i in transcripts])
  length_factors = length / min(length)

  # Transcripts counts estimation
  def expectation(transcripts, reads, abundance):
    counts = np.zeros(len(transcripts))
    for i, read in enumerate(reads):
      equivalence_class = np.array([
          read in transcript for transcript in transcripts
      ]) # From which transcript our read might be?
      counts[equivalence_class] += (
          abundance[equivalence_class] / sum(abundance[equivalence_class])
      ) # Add fraction of the read to the transcripts according their abundances

    counts = np.round(counts, 3)
    return counts

  # Abundances maximization
  def maximization(counts, length_factors):
    abundance = (counts / sum(counts)) / length_factors # Taking into account
    # length factor
    abundance = abundance / sum(abundance)
    return abundance

  # Initianion
  abundance = np.ones(len(transcripts)) / len(transcripts) # Equal amount
  counts = expectation(transcripts, reads, abundance)

  # EM steps
  prev_counts = np.zeros(len(transcripts))
  step = 0
  while list(prev_counts) != list(counts): # Until convergence
    step += 1
    prev_counts = counts.copy()
    abundance = maximization(counts, length_factors)
    counts = expectation(transcripts, reads, abundance)

  return (abundance, counts, step)

And let's test our algorithm!

In [4]:
abundance, counts, step = EM(transcripts, reads)

print("Differences in abundance (in %):")
print((abundance - abundance_ground_truth) * 100)

Differences in abundance (in %):
[-0.30047529 -1.00559749 -0.57102735  1.21407655  0.66302358]


And now, let's try to do the same calculation with `kallisto`. At the beginning, let's generate pseudo-transcripts and pseudo-reads.

In [5]:
read_length = 50

random.seed(seed)
blocks_sequences = dict(zip(set("".join(transcripts)), [
    "".join([np.random.choice(list("ATGC")) for i in range(read_length)])
    for j in set("".join(transcripts))
]))

transcript_sequences = dict(zip(transcripts, [
    "".join([
        blocks_sequences[block] for block in transcript
    ]) for transcript in transcripts
]))

with open("reference.fasta", "w") as f:
  for transcript in transcript_sequences:
    f.write(f">{transcript}\n{transcript_sequences[transcript]}\n")

with open("reads.fastq", "w") as f:
  for i, read in enumerate(reads):
    f.write(f"@Read_{i}\n{blocks_sequences[read]}\n+\n{'I' * read_length}\n")

And let's launch `kallisto`. Firstly, let's install the tool.

In [6]:
!git clone https://github.com/pachterlab/kallisto.git
!apt-get install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make

Cloning into 'kallisto'...
remote: Enumerating objects: 8586, done.
remote: Counting objects: 100% (1602/1602), done.
remote: Compressing objects: 100% (543/543), done.
remote: Total 8586 (delta 1174), reused 1302 (delta 1057), pack-reused 6984
Receiving objects: 100% (8586/8586), 9.29 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (5692/5692), done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4

Second, let's generate the reference index.

In [7]:
!./kallisto/build/src/kallisto index -i reference.idx reference.fasta


[build] loading fasta file reference.fasta
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 1090
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 288
CompactedDBG::filter(): Processed 1400 k-mers in 5 reads
CompactedDBG::filter(): Found 1079 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 9
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 12 unitigs
CompactedDBG::construct(): After split (1/1): 12 unitigs
CompactedDBG::construct()

After, let's estimate abundances.

In [8]:
!./kallisto/build/src/kallisto quant -i reference.idx -o results --single -l 50 -s 1 reads.fastq


[quant] fragment length distribution is truncated gaussian with mean = 50, sd = 1
[index] k-mer length: 31
[index] number of targets: 5
[index] number of k-mers: 1,080
[quant] running in single-end mode
[quant] will process file 1: reads.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 1,503 reads, 1,426 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds



And finally, let's compare the results.

In [9]:
import pandas as pd

kallisto_results = pd.read_csv("results/abundance.tsv", sep="\t")
kallisto_abundances = kallisto_results.tpm / kallisto_results.tpm.sum()

pd.DataFrame({
    "Difference true — kallisto (%)": list((abundance_ground_truth - kallisto_abundances) * 100),
    "Difference true — our (%)": list((abundance_ground_truth - abundance) * 100)
}, index=transcript_sequences.keys())

,Difference true — kallisto (%),Difference true — our (%)
ABCDEZQ,2.508253,0.300475
DEFHI,1.397121,1.005597
GHIL,1.836073,0.571027
KLONPQST,-3.345079,-1.214077
PQRSTUW,-2.396368,-0.663024


# Hometask

## Task 1

Please, provide an example where even EM-based algorithm estimates expression wrong. Describe in which exact cases it might happens.

Where EM Can Go Wrong

1. Initialization Sensitivity: The starting values of the parameters (the initial guess) can significantly influence the final estimates.  A poor initial guess might lead the algorithm to a local maximum, rather than the global maximum, of the likelihood function.

2. Non-Identifiability: If the model is not identifiable, meaning multiple parameter combinations can produce the same observed data, EM may not converge to the "true" values. This can occur when there are more parameters than observed data points or when the model has strong dependencies between variables.

3. Local Maxima: The likelihood function may have multiple local maxima. EM will converge to one of these maxima, which might not be the global maximum. This can lead to inaccurate estimates, particularly if the initial guess is close to a local maximum.

4. Assumptions of Model:  EM relies on specific assumptions about the underlying data distribution and the relationships between variables. If these assumptions are violated, the estimates can be inaccurate.

## Task 2

Using [recount3](https://rna.recount.bio/), find any available RNA-Seq dataset (only one sample will be enough) with already estimated expressions at the level of gene counts. Re-estimate the expressions with `kallisto` and compare the results (at the gene level).

Can't find avalible raw data :(